# Import

In [1]:
import time

In [2]:
import pandas as pd

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline

from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

/home/container_user/.cache/pypoetry/virtualenvs/hansoldeco3-QckXwucX-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import transformers
import accelerate
print(transformers.__version__)
print(accelerate.__version__)

4.49.0
1.4.0


In [4]:
print("CUDA 사용 가능:", torch.cuda.is_available())
print("GPU 개수:", torch.cuda.device_count())
print("사용 중인 GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU 사용 중")

CUDA 사용 가능: True
GPU 개수: 1
사용 중인 GPU: NVIDIA GeForce RTX 3080


# Data Load & Pre-processing

In [5]:
path_train = "/workspace/Storage/hansoldeco3/Data/raw/train.csv"
path_test = "/workspace/Storage/hansoldeco3/Data/raw/test.csv"
sample_idx = "/workspace/Storage/hansoldeco3/Data/sample/sample_v1.csv"

In [6]:
train = pd.read_csv(path_train, encoding = 'utf-8-sig')
test = pd.read_csv(path_test, encoding = 'utf-8-sig')
sample_idx = pd.read_csv(sample_idx)

In [7]:
# val 데이터 선택
test = train[train["ID"].isin(sample_idx["sample_v1"])]
# train 데이터 선택 (val에 포함되지 않은 나머지)
train = train[~train["ID"].isin(sample_idx["sample_v1"])]

In [8]:
# 데이터 전처리
train['공사종류(대분류)'] = train['공사종류'].str.split(' / ').str[0]
train['공사종류(중분류)'] = train['공사종류'].str.split(' / ').str[1]
train['공종(대분류)'] = train['공종'].str.split(' > ').str[0]
train['공종(중분류)'] = train['공종'].str.split(' > ').str[1]
train['사고객체(대분류)'] = train['사고객체'].str.split(' > ').str[0]
train['사고객체(중분류)'] = train['사고객체'].str.split(' > ').str[1]

test['공사종류(대분류)'] = test['공사종류'].str.split(' / ').str[0]
test['공사종류(중분류)'] = test['공사종류'].str.split(' / ').str[1]
test['공종(대분류)'] = test['공종'].str.split(' > ').str[0]
test['공종(중분류)'] = test['공종'].str.split(' > ').str[1]
test['사고객체(대분류)'] = test['사고객체'].str.split(' > ').str[0]
test['사고객체(중분류)'] = test['사고객체'].str.split(' > ').str[1]

In [9]:
# 훈련 데이터 통합 생성
combined_training_data = train.apply(
    lambda row: {
        "question": (
            f"공사종류 대분류 '{row['공사종류(대분류)']}', 중분류 '{row['공사종류(중분류)']}' 공사 중 "
            f"공종 대분류 '{row['공종(대분류)']}', 중분류 '{row['공종(중분류)']}' 작업에서 "
            f"사고객체 '{row['사고객체(대분류)']}'(중분류: '{row['사고객체(중분류)']}')와 관련된 사고가 발생했습니다. "
            f"작업 프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}'입니다. "
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
        "answer": row["재발방지대책 및 향후조치계획"]
    },
    axis=1
)

# DataFrame으로 변환
combined_training_data = pd.DataFrame(list(combined_training_data))

In [10]:
# 테스트 데이터 통합 생성
combined_test_data = test.apply(
    lambda row: {
        "question": (
            f"공사종류 대분류 '{row['공사종류(대분류)']}', 중분류 '{row['공사종류(중분류)']}' 공사 중 "
            f"공종 대분류 '{row['공종(대분류)']}', 중분류 '{row['공종(중분류)']}' 작업에서 "
            f"사고객체 '{row['사고객체(대분류)']}'(중분류: '{row['사고객체(중분류)']}')와 관련된 사고가 발생했습니다. "
            f"작업 프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}'입니다. "
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        )
    },
    axis=1
)

# DataFrame으로 변환
combined_test_data = pd.DataFrame(list(combined_test_data))

# Model import

In [11]:
from vllm import LLM, SamplingParams

model_id = "NCSOFT/Llama-VARCO-8B-Instruct"

# tokenizer = AutoTokenizer.from_pretrained(model_id)

# vLLM에서 bitsandbytes 8bit 적용
llm = LLM(
    model=model_id,
    quantization="bitsandbytes",
    load_format="bitsandbytes",  # 이 옵션 추가
    dtype=torch.float16,
    gpu_memory_utilization=0.9  # GPU 메모리 활용율 조정 가능
)


2025-03-05 11:08:34,663	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 03-05 11:08:35 __init__.py:207] Automatically detected platform cuda.
WARNING 03-05 11:08:36 config.py:2448] Casting torch.bfloat16 to torch.float16.
INFO 03-05 11:08:42 config.py:549] This model supports multiple tasks: {'reward', 'embed', 'generate', 'score', 'classify'}. Defaulting to 'generate'.
WARNING 03-05 11:08:42 config.py:628] bitsandbytes quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 03-05 11:08:43 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='NCSOFT/Llama-VARCO-8B-Instruct', speculative_config=None, tokenizer='NCSOFT/Llama-VARCO-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.BITSANDBYTES, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=bitsandbytes, enfor

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:01,  2.12it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:04<00:05,  2.58s/it]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:06<00:02,  2.40s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:08<00:00,  2.07s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:08<00:00,  2.07s/it]



INFO 03-05 11:08:55 model_runner.py:1115] Loading model weights took 5.3129 GB
INFO 03-05 11:08:58 worker.py:267] Memory profiling takes 3.16 seconds
INFO 03-05 11:08:58 worker.py:267] the current vLLM instance can use total_gpu_memory (10.00GiB) x gpu_memory_utilization (0.90) = 9.00GiB
INFO 03-05 11:08:58 worker.py:267] model weights take 5.31GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 1.23GiB; the rest of the memory reserved for KV Cache is 2.40GiB.
INFO 03-05 11:08:58 executor_base.py:111] # cuda blocks: 1227, # CPU blocks: 2048
INFO 03-05 11:08:58 executor_base.py:116] Maximum concurrency for 8192 tokens per request: 2.40x
INFO 03-05 11:08:59 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utiliz

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:25<00:00,  1.37it/s]

INFO 03-05 11:09:25 model_runner.py:1562] Graph capturing finished in 25 secs, took 0.70 GiB
INFO 03-05 11:09:25 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 30.11 seconds


# Vector store 생성

In [ ]:
# Train 데이터 준비
train_questions_prevention = combined_training_data['question'].tolist()
train_answers_prevention = combined_training_data['answer'].tolist()

train_documents = [
    f"Q: {q1}\nA: {a1}"
    for q1, a1 in zip(train_questions_prevention, train_answers_prevention)
]

# 임베딩 생성
embedding_model_name = "jhgan/ko-sbert-nli"  # 임베딩 모델 선택
embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)

# 벡터 스토어에 문서 추가
vector_store = FAISS.from_texts(train_documents, embedding)

# Retriever 정의
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})


/tmp/ipykernel_872713/3768745628.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)


# RAG chain 생성

In [ ]:
from langchain_core.runnables import Runnable
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.runnables import Runnable, RunnableLambda


# vLLM을 LangChain 호환 `Runnable`로 감싸는 클래스
class VLLMRunner(Runnable, BaseModel):
    llm: LLM

    class Config:
        arbitrary_types_allowed = True  # ✅ vllm.LLM 객체 허용

    def invoke(self, input_text: Any, config=None, **kwargs) -> str:
        """vLLM을 호출하여 응답을 반환하는 함수"""
        # ✅ 입력이 딕셔너리라면 문자열로 변환
        if isinstance(input_text, dict):
            input_text = input_text.get("question", "")

        input_text = str(input_text)  # 문자열로 강제 변환

        sampling_params = SamplingParams(
            temperature=kwargs.get("temperature", 0.7),
            max_tokens=kwargs.get("max_tokens", 512),
            stop=kwargs.get("stop", None),
        )
        outputs = self.llm.generate([input_text], sampling_params)
        return outputs[0].outputs[0].text  # 생성된 텍스트 반환

In [ ]:
# ✅ vLLM을 `RunnableLambda`로 감싸서 LangChain과 호환되도록 수정
wrapped_llm = RunnableLambda(lambda x, **kwargs: VLLMRunner(llm=llm).invoke(x, **kwargs))


In [ ]:

prompt_template = """
### 지침: 당신은 건설 안전 전문가입니다.
질문에 대한 답변을 핵심 내용만 요약하여 간략하게 작성하세요.
- 서론, 배경 설명 또는 추가 설명을 절대 포함하지 마세요.
- 다음과 같은 조치를 취할 것을 제안합니다: 와 같은 내용을 포함하지 마세요.

{context}

### 질문:
{question}

[/INST]

"""


# 커스텀 프롬프트 생성
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)


# RAG 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=wrapped_llm,
    chain_type="stuff",  # 단순 컨텍스트 결합 방식 사용
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}  # 커스텀 프롬프트 적용
)

# Inference

In [48]:
# 테스트 실행 및 결과 저장
test_results = []

print("테스트 실행 시작... 총 테스트 샘플 수:", len(combined_test_data))

for idx, row in combined_test_data.iterrows():

    t = time.time()
    print(f"\n[샘플 {idx + 1}/{len(combined_test_data)}] 진행 중...")
    print(row['question'])
    # RAG 체인 호출 및 결과 생성
    prevention_result = qa_chain.invoke(
        row['question'],
        # llm_kwargs={"max_new_tokens": 50, "use_cache": True, "do_sample": False}
        )

    # 결과 저장
    result_text = prevention_result['result']
    test_results.append(result_text)
    print(f"iter {idx} 완료 : {time.time()-t}초")
print("\n테스트 실행 완료! 총 결과 수:", len(test_results))

테스트 실행 시작... 총 테스트 샘플 수: 100

[샘플 1/100] 진행 중...
공사종류 대분류 '토목', 중분류 '도로' 공사 중 공종 대분류 '토목', 중분류 '도로 및 포장공사' 작업에서 사고객체 '건설공구'(중분류: '공구류')와 관련된 사고가 발생했습니다. 작업 프로세스는 '설치작업'이며, 사고 원인은 '거푸집 설치를 위해 함마드릴 사용 중 손목이 뒤틀리며 통증'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.72s/it, est. speed input: 245.62 toks/s, output: 68.66 toks/s]


iter 0 완료 : 4.746534824371338초

[샘플 2/100] 진행 중...
공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류 '철근콘크리트공사' 작업에서 사고객체 '건설자재'(중분류: '철근')와 관련된 사고가 발생했습니다. 작업 프로세스는 '설치작업'이며, 사고 원인은 '철근 작업자가 PIT층에서 철근 절곡기를 이용한 절단작업 중 절단된 철근(약 3cm)이 인중으로 튀면서 살짝 찢어짐'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?


Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.35s/it, est. speed input: 288.11 toks/s, output: 67.42 toks/s]


...

iter 97 완료 : 7.5757293701171875초

[샘플 99/100] 진행 중...
공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류 '철골공사' 작업에서 사고객체 '기타'(중분류: '기타')와 관련된 사고가 발생했습니다. 작업 프로세스는 '이동'이며, 사고 원인은 '1층바닥에서 미끄러짐'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?


Processed prompts: 100%|██████████| 1/1 [00:07<00:00,  7.48s/it, est. speed input: 129.67 toks/s, output: 68.45 toks/s]


iter 98 완료 : 7.504142761230469초

[샘플 100/100] 진행 중...
공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '기타', 중분류 '기타' 작업에서 사고객체 '기타'(중분류: '기타')와 관련된 사고가 발생했습니다. 작업 프로세스는 '이동'이며, 사고 원인은 '공사작업끝나고 술한잔 먹고 내려오다가 계단에서 미끄러져서 다침'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?


Processed prompts: 100%|██████████| 1/1 [00:08<00:00,  8.87s/it, est. speed input: 133.93 toks/s, output: 47.91 toks/s]

iter 99 완료 : 8.896803140640259초

테스트 실행 완료! 총 결과 수: 100


In [52]:
pd.DataFrame(test_results, columns = ['answer']).to_csv('./vllm_result.csv')

# Submission

In [ ]:
from sentence_transformers import SentenceTransformer

embedding_model_name = "jhgan/ko-sbert-sts"
embedding = SentenceTransformer(embedding_model_name)

# 문장 리스트를 입력하여 임베딩 생성
pred_embeddings = embedding.encode(test_results)
print(pred_embeddings.shape)  # (샘플 개수, 768)

(964, 768)


In [ ]:
submission = pd.read_csv('/workspace/Storage/hansoldeco3/Data/sample_submission.csv', encoding = 'utf-8-sig')

# 최종 결과 저장
submission.iloc[:,1] = test_results
submission.iloc[:,2:] = pred_embeddings
submission.head()

# 최종 결과를 CSV로 저장
submission.to_csv('./baseline_submission.csv', index=False, encoding='utf-8-sig')